# Batch Translation with S3 Parquet Files

This notebook demonstrates how to perform large-scale batch translation using SentinelTranslate's S3-based parquet processing API.

## What You'll Learn

- Create parquet files with multilingual data
- Upload data to AWS S3
- Submit batch translation jobs via API
- Monitor job progress and status
- Download and analyze translated results
- Handle errors and retry strategies
- Optimize for throughput and cost

## Prerequisites

**Python packages**:
```bash
pip install pandas pyarrow boto3 requests matplotlib
```

**AWS credentials** (set as environment variables):
```bash
export AWS_ACCESS_KEY_ID='your-key'
export AWS_SECRET_ACCESS_KEY='your-secret'
export AWS_DEFAULT_REGION='us-east-1'
```

**Services running**:
```bash
docker-compose up -d
```

**Estimated time**: 30-60 minutes

## Step 1: Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import boto3
import requests
import time
import json
from datetime import datetime, timedelta
from typing import Dict, Any

# Configuration
BATCH_API_URL = "http://localhost:8090"
S3_BUCKET = "my-translation-bucket"  # Update with your bucket name
S3_INPUT_PREFIX = "input/"
S3_OUTPUT_PREFIX = "output/"

print("✅ Imports successful")
print(f"📦 Batch API: {BATCH_API_URL}")
print(f"🪣 S3 Bucket: {S3_BUCKET}")

## Step 2: Verify Services are Running

In [ ]:
# Check batch API health
try:
    response = requests.get(f"{BATCH_API_URL}/health", timeout=5)
    if response.status_code == 200:
        print("✅ Batch API is healthy")
    else:
        print(f"⚠️  Batch API returned status {response.status_code}")
except Exception as e:
    print(f"❌ Batch API not reachable: {e}")
    print("   Please start services: docker-compose up -d")

## Step 3: Create Sample Data

Let's create a realistic dataset with multilingual customer reviews.

In [ ]:
# Create sample multilingual data
np.random.seed(42)

# French reviews
french_reviews = [
    "Excellent produit, très satisfait de mon achat.",
    "La qualité est médiocre pour le prix.",
    "Livraison rapide et service client réactif.",
    "Je recommande vivement cette entreprise.",
    "Le produit ne correspond pas à la description.",
    "Très bon rapport qualité-prix.",
    "Déçu par la qualité du matériau.",
    "Service après-vente impeccable.",
    "Le délai de livraison était trop long.",
    "Produit conforme à mes attentes."
]

# Spanish reviews
spanish_reviews = [
    "Excelente producto, muy satisfecho con la compra.",
    "La calidad es mediocre para el precio.",
    "Entrega rápida y servicio al cliente receptivo.",
    "Recomiendo encarecidamente esta empresa.",
    "El producto no coincide con la descripción.",
    "Muy buena relación calidad-precio.",
    "Decepcionado con la calidad del material.",
    "Servicio postventa impecable.",
    "El tiempo de entrega fue demasiado largo.",
    "Producto conforme a mis expectativas."
]

# German reviews
german_reviews = [
    "Ausgezeichnetes Produkt, sehr zufrieden mit dem Kauf.",
    "Die Qualität ist mittelmäßig für den Preis.",
    "Schnelle Lieferung und reaktionsschneller Kundenservice.",
    "Ich empfehle dieses Unternehmen wärmstens.",
    "Das Produkt entspricht nicht der Beschreibung.",
    "Sehr gutes Preis-Leistungs-Verhältnis.",
    "Enttäuscht von der Materialqualität.",
    "Einwandfreier Kundenservice.",
    "Die Lieferzeit war zu lang.",
    "Produkt entspricht meinen Erwartungen."
]

# Combine all reviews
all_reviews = (
    [(text, 'fr') for text in french_reviews] +
    [(text, 'es') for text in spanish_reviews] +
    [(text, 'de') for text in german_reviews]
)

# Create DataFrame
df = pd.DataFrame({
    'review_id': range(1, len(all_reviews) + 1),
    'text': [r[0] for r in all_reviews],
    'source_lang': [r[1] for r in all_reviews],
    'rating': np.random.randint(1, 6, len(all_reviews)),
    'product_category': np.random.choice(
        ['Electronics', 'Clothing', 'Home', 'Books', 'Sports'],
        len(all_reviews)
    ),
    'created_at': [
        datetime.now() - timedelta(days=np.random.randint(0, 365))
        for _ in range(len(all_reviews))
    ]
})

print(f"✅ Created sample dataset with {len(df)} reviews")
print(f"\nLanguage distribution:")
print(df['source_lang'].value_counts())
print(f"\nSample data:")
print(df.head(10))

## Step 4: Save Data to Parquet

Parquet is a columnar storage format that's efficient for large datasets.

In [ ]:
# Save to parquet files (one per language for this example)
parquet_files = {}

for lang in df['source_lang'].unique():
    lang_df = df[df['source_lang'] == lang]
    filename = f'reviews_{lang}.parquet'
    lang_df.to_parquet(filename, index=False, engine='pyarrow')
    parquet_files[lang] = filename
    print(f"✅ Saved {filename} ({len(lang_df)} rows)")

# Also save combined file
combined_file = 'reviews_combined.parquet'
df.to_parquet(combined_file, index=False, engine='pyarrow')
print(f"✅ Saved {combined_file} ({len(df)} rows)")

## Step 5: Upload to S3

Upload parquet files to S3 for processing.

In [ ]:
# Initialize S3 client
s3_client = boto3.client('s3')

# Create bucket if it doesn't exist (optional)
try:
    s3_client.head_bucket(Bucket=S3_BUCKET)
    print(f"✅ Bucket {S3_BUCKET} exists")
except:
    print(f"Creating bucket {S3_BUCKET}...")
    s3_client.create_bucket(Bucket=S3_BUCKET)
    print(f"✅ Created bucket {S3_BUCKET}")

# Upload files
s3_files = {}
for lang, local_file in parquet_files.items():
    s3_key = f"{S3_INPUT_PREFIX}{local_file}"
    s3_client.upload_file(local_file, S3_BUCKET, s3_key)
    s3_files[lang] = s3_key
    print(f"✅ Uploaded {local_file} to s3://{S3_BUCKET}/{s3_key}")

# Upload combined file
combined_s3_key = f"{S3_INPUT_PREFIX}{combined_file}"
s3_client.upload_file(combined_file, S3_BUCKET, combined_s3_key)
print(f"✅ Uploaded {combined_file} to s3://{S3_BUCKET}/{combined_s3_key}")

## Step 6: Helper Functions for Batch Translation

In [ ]:
def submit_batch_job(s3_bucket: str, s3_key: str, text_column: str,
                     source_lang: str, target_lang: str = 'en',
                     output_s3_key: str = None, id_column: str = None) -> Dict[str, Any]:
    """Submit a batch translation job."""
    payload = {
        's3_bucket': s3_bucket,
        's3_key': s3_key,
        'text_column': text_column,
        'source_lang': source_lang,
        'target_lang': target_lang
    }
    
    if output_s3_key:
        payload['output_s3_key'] = output_s3_key
    if id_column:
        payload['id_column'] = id_column
    
    try:
        response = requests.post(f"{BATCH_API_URL}/batch/translate", json=payload, timeout=30)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Failed to submit job: {e}")
        return None

def check_job_status(job_id: str) -> Dict[str, Any]:
    """Check status of a batch job."""
    try:
        response = requests.get(f"{BATCH_API_URL}/batch/status/{job_id}", timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Failed to check status: {e}")
        return None

def wait_for_completion(job_id: str, poll_interval: int = 5, max_wait: int = 600) -> Dict[str, Any]:
    """Poll until job completes or times out."""
    start_time = time.time()
    
    while (time.time() - start_time) < max_wait:
        status = check_job_status(job_id)
        
        if not status:
            time.sleep(poll_interval)
            continue
        
        print(f"Job {job_id[:8]}... status: {status['status']}")
        
        if status['status'] in ['SUCCESS', 'FAILURE']:
            return status
        
        time.sleep(poll_interval)
    
    print(f"⏱️  Timeout after {max_wait}s")
    return None

print("✅ Helper functions defined")

## Step 7: Submit Batch Translation Jobs

Submit a job for each language.

In [ ]:
# Submit jobs for each language
jobs = {}

for lang, s3_key in s3_files.items():
    output_key = f"{S3_OUTPUT_PREFIX}reviews_{lang}_translated.parquet"
    
    print(f"\nSubmitting job for {lang}...")
    result = submit_batch_job(
        s3_bucket=S3_BUCKET,
        s3_key=s3_key,
        text_column='text',
        source_lang=lang,
        target_lang='en',
        output_s3_key=output_key,
        id_column='review_id'
    )
    
    if result:
        jobs[lang] = {
            'job_id': result['job_id'],
            'output_key': output_key,
            'submitted_at': datetime.now()
        }
        print(f"✅ Job submitted: {result['job_id']}")
    else:
        print(f"❌ Failed to submit job for {lang}")

print(f"\n📊 Submitted {len(jobs)} jobs")

## Step 8: Monitor Job Progress

Wait for all jobs to complete.

In [ ]:
# Wait for all jobs to complete
print("Waiting for jobs to complete...\n")

results = {}
for lang, job_info in jobs.items():
    job_id = job_info['job_id']
    print(f"\nMonitoring {lang} job ({job_id[:8]}...)")
    
    final_status = wait_for_completion(job_id, poll_interval=5, max_wait=300)
    
    if final_status and final_status['status'] == 'SUCCESS':
        print(f"✅ {lang} translation completed successfully")
        print(f"   Output: {final_status['result']['output_location']}")
        print(f"   Total rows: {final_status['result']['total_rows']}")
        print(f"   Successful: {final_status['result']['successful_translations']}")
        print(f"   Failed: {final_status['result']['failed_rows']}")
        results[lang] = final_status
    else:
        print(f"❌ {lang} translation failed or timed out")
        if final_status:
            print(f"   Error: {final_status.get('error', 'Unknown')}")

print(f"\n🎉 {len(results)} of {len(jobs)} jobs completed successfully")

## Step 9: Download and Analyze Results

In [ ]:
# Download translated files
translated_dfs = {}

for lang, result in results.items():
    output_location = result['result']['output_location']
    # Parse s3://bucket/key
    bucket, key = output_location.replace('s3://', '').split('/', 1)
    
    local_file = f'reviews_{lang}_translated.parquet'
    
    print(f"Downloading {lang} results...")
    s3_client.download_file(bucket, key, local_file)
    
    # Read parquet
    df_translated = pd.read_parquet(local_file)
    translated_dfs[lang] = df_translated
    
    print(f"✅ Downloaded {local_file} ({len(df_translated)} rows)")
    print(f"   Columns: {list(df_translated.columns)}")
    print(f"\nSample translations:")
    print(df_translated[['text', 'translation']].head(3))
    print()

## Step 10: Quality Analysis

Analyze translation quality and performance.

In [ ]:
# Combine all translated data
all_translated = pd.concat(translated_dfs.values(), ignore_index=True)

print(f"📊 Translation Summary\n")
print(f"Total rows processed: {len(all_translated)}")
print(f"Successful translations: {len(all_translated[all_translated['translation'].notna()])}")
print(f"Failed translations: {len(all_translated[all_translated['translation'].isna()])}")

# Language breakdown
print(f"\nBy language:")
for lang in all_translated['source_lang'].unique():
    lang_df = all_translated[all_translated['source_lang'] == lang]
    success_count = len(lang_df[lang_df['translation'].notna()])
    print(f"  {lang}: {success_count}/{len(lang_df)} successful")

# Average text length
all_translated['text_length'] = all_translated['text'].str.len()
all_translated['translation_length'] = all_translated['translation'].str.len()

print(f"\nText statistics:")
print(f"  Average source length: {all_translated['text_length'].mean():.0f} chars")
print(f"  Average translation length: {all_translated['translation_length'].mean():.0f} chars")
print(f"  Length ratio: {(all_translated['translation_length'] / all_translated['text_length']).mean():.2f}")

## Step 11: Visualization (Optional)

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    # Plot translation length comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Length distribution
    all_translated.boxplot(column=['text_length', 'translation_length'], ax=ax1)
    ax1.set_title('Text Length Distribution')
    ax1.set_ylabel('Characters')
    ax1.set_xticklabels(['Source', 'Translation'])
    ax1.grid(True, alpha=0.3)
    
    # Success rate by language
    success_by_lang = all_translated.groupby('source_lang').apply(
        lambda x: len(x[x['translation'].notna()]) / len(x) * 100
    )
    success_by_lang.plot(kind='bar', ax=ax2, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax2.set_title('Success Rate by Language')
    ax2.set_ylabel('Success Rate (%)')
    ax2.set_xlabel('Source Language')
    ax2.set_ylim([0, 100])
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Install matplotlib for visualizations: pip install matplotlib")

## Summary

In this notebook, you learned:

✅ How to prepare multilingual data in parquet format  
✅ Upload data to AWS S3  
✅ Submit batch translation jobs via API  
✅ Monitor job progress and handle completion  
✅ Download and analyze translated results  
✅ Measure translation quality and performance  

## Next Steps

1. **Scale up**: Process larger datasets (10K+ rows)
2. **Optimize**: Tune worker count for better throughput
3. **Integrate**: Add batch translation to your data pipelines
4. **Monitor**: Set up alerts for job failures
5. **Quality**: Implement custom quality checks

## Additional Resources

- [Batch Translation README](README.md)
- [SentinelTranslate Documentation](../../README.md)
- [AWS S3 Documentation](https://docs.aws.amazon.com/s3/)
- [Parquet Format](https://parquet.apache.org/)

---

**Questions?** Open an issue on GitHub or check the main documentation.